# 1. Case study: refactory Strategy

## Classic Strategy

In [1]:
# Ex 6-1 Implementation Order class with pluggable discount strategies.

from abc import ABC, abstractclassmethod

In [2]:
from collections import namedtuple
Customer = namedtuple('Customer', 'name fidelity')

In [3]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity
    
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [7]:
class Promotion(ABC): # the Strategy: an Abstract Base Class

    @abstractclassmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

In [8]:
class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [9]:
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [10]:
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}

        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [11]:
# Ex 6-2 Sample Usage of Order class with different promotions applied

In [12]:
joe = Customer('Jonh Doe', 0)
ann = Customer('Ann Smith', 1100)

In [13]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)
       ]

In [14]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [15]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [16]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [17]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [18]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [19]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [20]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-oriented Strategy

In [21]:
# Ex 6-3 Order class with discount strategies implemented as functions
from collections import namedtuple
Customer = namedtuple('Customer', 'name fidelity')

In [22]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity
    
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [23]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [24]:
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
        return discount

In [25]:
def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

- Strategy objects often make good flyweights
- A flyweight is a shared object that can be used in multiple contexts simultaneously

## Choosing the best strategy: simple approach

In [26]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

## Finding Strategies in a module
- globals(): Return a dictionary representing the current global symbol table.

In [27]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

In [32]:
promos

[<function __main__.large_order_promo>,
 <function __main__.bulk_item_promo>,
 <function __main__.fidelity_promo>]

In [35]:
import inspect
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

# 2. Command